In [ ]:
import nltk
import random
import time

import pickle
#wrapper of Sklearn Around nltk classifiers
from nltk.classify.scikitlearn import SklearnClassifier

#importing various sckitlearn clsssifiers and testing them on a training data
from sklearn.naive_bayes import MultinomialNB ,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC,NuSVC
from nltk.classify import ClassifierI
from statistics import mode

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [ ]:
import string
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/MyDrive/Text Mining')
os.getcwd()

'/content/drive/MyDrive/Text Mining'

# Import Data

In [ ]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

test2 = pd.read_csv('dataset/IMDB Dataset.csv', header=0, names=['reviews_content', 'category'])
test2 = test2.iloc[:500]

# test2 = pd.read_csv('twitter_validation.csv', header=None)
# test2 = test2.iloc[1:, 2:]
# test2 = test2.rename(columns={2: 'category', 3: 'reviews_content'})

print('train head: ', train.shape)
print(train.head())
print('\n\ntest head: ', test.shape)
print(test.head())
print('\n\ntest2 head: ', test2.shape)
print(test2.head())

train head:  (1500, 2)
                                     reviews_content  category
0  airplane ! is considered among many to be the ...  positive
1  you've got to love disney . \nno matter what t...  positive
2   " the tailor of panama " is a different kind ...  positive
3  the characters in jonathan lynn's " the whole ...  negative
4  vikings v . bears ? \nno , this isn't the line...  negative


test head:  (500, 1)
                                     reviews_content
0  towards the middle of " the sweet hereafter , ...
1  wild things is a suspenseful thriller starring...
2  hong kong cinema has been going through a bad ...
3  while alex browning ( devon sawa ) waits at jf...
4  sometimes i find 19th century british costume ...


test2 head:  (500, 2)
                                     reviews_content  category
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spen

# Preprocess and Feature Engineering

In [ ]:
train['reviews_content'].dropna(inplace=True)
test['reviews_content'].dropna(inplace=True)
test2['reviews_content'].dropna(inplace=True)

print('train shape: ', train.shape)
print('test shape: ', test.shape)
print('test2 shape: ', test2.shape)

train shape:  (1500, 2)
test shape:  (500, 1)
test2 shape:  (500, 2)


In [ ]:
def preprocess(text: str) -> list:
    corpus = str(text).lower() # lowercase
    corpus = re.sub('[^a-zA-Z]+',' ', corpus).strip() # remove special characters
    tokens = word_tokenize(corpus) # tokenize
    words = [token for token in tokens if token not in stop_words] # remove stopwords
    # words = [lemmatizer.lemmatize(word) for word in words] # lemmatize
    return words

In [ ]:
with open('dataset/positive-words.txt', 'rb') as f:
    positive_words = f.read().decode('latin-1').split()

with open('dataset/negative-words.txt', 'rb') as f:
    negative_words = f.read().decode('latin-1').split()

In [ ]:
le = LabelEncoder()

In [ ]:
preprocess_train = train['reviews_content'].apply(preprocess)
preprocess_test = test['reviews_content'].apply(preprocess)
preprocess_test2 = test2['reviews_content'].apply(preprocess)

train['preprocessed'] = preprocess_train
train['total_len'] = train['preprocessed'].apply(len)
train['pos_count'] = train['preprocessed'].apply(lambda x: len([word for word in x if word in positive_words]))
train['neg_count'] = train['preprocessed'].apply(lambda x: len([word for word in x if word in negative_words]))
train['category'] = le.fit_transform(train['category'])

test['preprocessed'] = preprocess_test
test['total_len'] = test['preprocessed'].apply(len)
test['pos_count'] = test['preprocessed'].apply(lambda x: len([word for word in x if word in positive_words]))
test['neg_count'] = test['preprocessed'].apply(lambda x: len([word for word in x if word in negative_words]))

test2['preprocessed'] = preprocess_test2
test2['total_len'] = test2['preprocessed'].apply(len)
test2['pos_count'] = test2['preprocessed'].apply(lambda x: len([word for word in x if word in positive_words]))
test2['neg_count'] = test2['preprocessed'].apply(lambda x: len([word for word in x if word in negative_words]))

In [ ]:
# using VADER
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sent = SentimentIntensityAnalyzer()

polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in train['reviews_content']]
train['sentiment'] = polarity

polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in test['reviews_content']]
test['sentiment'] = polarity

polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in test2['reviews_content']]
test2['sentiment'] = polarity

In [ ]:
train['preprocessed_final'] = train['preprocessed'].apply(' '.join)
test['preprocessed_final'] = test['preprocessed'].apply(' '.join)
test2['preprocessed_final'] = test2['preprocessed'].apply(' '.join)

In [ ]:
train.head()

,reviews_content,category,preprocessed,total_len,pos_count,neg_count,sentiment,preprocessed_final
0,airplane ! is considered among many to be the ...,1,"[airplane, considered, among, many, epitome, s...",312,18,25,-0.66,airplane considered among many epitome satire ...
1,you've got to love disney . \nno matter what t...,1,"[got, love, disney, matter, serve, guaranteed,...",629,57,28,1.00,got love disney matter serve guaranteed succes...
2,""" the tailor of panama "" is a different kind ...",1,"[tailor, panama, different, kind, spy, movie, ...",464,51,25,1.00,tailor panama different kind spy movie despite...
3,"the characters in jonathan lynn's "" the whole ...",0,"[characters, jonathan, lynn, whole, nine, yard...",514,38,35,1.00,characters jonathan lynn whole nine yards yet ...
4,"vikings v . bears ? \nno , this isn't the line...",0,"[vikings, v, bears, lineup, monday, night, foo...",308,26,28,-0.99,vikings v bears lineup monday night football r...


# Vectorizing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack

In [ ]:
scaler = StandardScaler()

In [ ]:
vectorizer = TfidfVectorizer(
    min_df=5,
    max_df=0.8,
    sublinear_tf=True,
    use_idf=True,
)

train_vectors = vectorizer.fit_transform(train['preprocessed_final'])
test_vectors = vectorizer.transform(test['preprocessed_final'])
test2_vectors = vectorizer.transform(test2['preprocessed_final'])

train_numerical_features = scaler.fit_transform(train[['total_len', 'pos_count', 'neg_count', 'sentiment']])
test_numerical_features = scaler.transform(test[['total_len', 'pos_count', 'neg_count', 'sentiment']])
test2_numerical_features = scaler.transform(test2[['total_len', 'pos_count', 'neg_count', 'sentiment']])

train_features = hstack([train_vectors, train_numerical_features])
test_features = hstack([test_vectors, test_numerical_features])
test2_features = hstack([test2_vectors, test2_numerical_features])

# Models

## SVM

In [ ]:
svm_linear = svm.SVC(kernel='linear')
t0 = time.time()

svm_linear.fit(train_features, train['category'])
t1 = time.time()

svm_linear_pred = le.inverse_transform(svm_linear.predict(test2_features))
t2 = time.time()

time_linear_train = t1-t0
time_linear_predict = t2-t1

print("Training time: %fs; Prediction time: %fs" % (time_linear_train, time_linear_predict))

Training time: 7.143004s; Prediction time: 0.804082s


In [ ]:
report = classification_report(test2['category'], svm_linear_pred, output_dict=True)

print('positive: ', report['positive'])
print('negative: ', report['negative'])

positive:  {'precision': 0.8963730569948186, 'recall': 0.729957805907173, 'f1-score': 0.8046511627906977, 'support': 237}
negative:  {'precision': 0.7915309446254072, 'recall': 0.9239543726235742, 'f1-score': 0.8526315789473684, 'support': 263}


In [ ]:
svm_final_pred = le.inverse_transform(svm_linear.predict(test_features))

## SGD Classifier

In [ ]:
sgd = SGDClassifier()
sgd.fit(train_features, train['category'])

SGDClassifier()

In [ ]:
sgd_pred = le.inverse_transform(sgd.predict(test2_features))

In [ ]:
report = classification_report(test2['category'], sgd_pred, output_dict=True)

print('positive: ', report['positive'])
print('negative: ', report['negative'])

positive:  {'precision': 0.7242524916943521, 'recall': 0.919831223628692, 'f1-score': 0.8104089219330856, 'support': 237}
negative:  {'precision': 0.9045226130653267, 'recall': 0.6844106463878327, 'f1-score': 0.7792207792207791, 'support': 263}


In [ ]:
sgd_final_pred = le.inverse_transform(sgd.predict(test_features))

## SVC

In [ ]:
from sklearn.svm import SVC

# Create a SVC classifier
SVC_classifier = SVC()

# Train the classifier
SVC_classifier.fit(train_features, train['category'])

SVC()

In [ ]:
# Predict the labels for the test data
SVC_pred = le.inverse_transform(SVC_classifier.predict(test2_features))

report = classification_report(test2['category'], SVC_pred, output_dict=True)

print('positive: ', report['positive'])
print('negative: ', report['negative'])

positive:  {'precision': 0.801980198019802, 'recall': 0.6835443037974683, 'f1-score': 0.7380410022779044, 'support': 237}
negative:  {'precision': 0.7483221476510067, 'recall': 0.8479087452471483, 'f1-score': 0.7950089126559715, 'support': 263}


In [ ]:
svc_final_pred = le.inverse_transform(SVC_classifier.predict(test_features))

## Logistic Regression

In [ ]:
# Create a Logistic Regression classifier
logreg = LogisticRegression()

# Train the classifier
logreg.fit(train_features, train['category'])

LogisticRegression()

In [ ]:
# Predict the labels for the test data
logreg_pred = le.inverse_transform(logreg.predict(test2_features))

In [ ]:
report = classification_report(test2['category'], logreg_pred, output_dict=True)

print('positive: ', report['positive'])
print('negative: ', report['negative'])

positive:  {'precision': 0.9315068493150684, 'recall': 0.5738396624472574, 'f1-score': 0.7101827676240209, 'support': 237}
negative:  {'precision': 0.7146892655367232, 'recall': 0.9619771863117871, 'f1-score': 0.8200972447325772, 'support': 263}


In [ ]:
logreg_final_pred = le.inverse_transform(logreg.predict(test_features))

## NuSVC

In [ ]:
nu_svc = NuSVC()

# Train the classifier
nu_svc.fit(train_features, train['category'])

# Predict the labels for the test data
nu_svc_pred = le.inverse_transform(nu_svc.predict(test2_features))

In [ ]:
report = classification_report(test2['category'], nu_svc_pred, output_dict=True)

print('positive: ', report['positive'])
print('negative: ', report['negative'])

positive:  {'precision': 0.8793103448275862, 'recall': 0.6455696202531646, 'f1-score': 0.7445255474452555, 'support': 237}
negative:  {'precision': 0.7423312883435583, 'recall': 0.9201520912547528, 'f1-score': 0.8217317487266553, 'support': 263}


In [ ]:
nusvc_final_pred = le.inverse_transform(nu_svc.predict(test_features))

## FairGBM

In [ ]:
%pip install fairgbm fairlearn scikit-learn hyperparameter-tuning seaborn matplotlib wget

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.1/234.1 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.0 MB/s eta 0:00:00
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=c4bf63f65ae48b47e2f4e8897c20da2dd38d4a39ccf3a46c3e4293ee5574a1e7
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [ ]:
from sklearn.model_selection import train_test_split

### Modify dataset for FairGBM

In [ ]:
X = train.drop(columns=['category', 'preprocessed', 'reviews_content'])
Y = train['category'].values
S = pd.cut(train['sentiment'], 3000, labels=False).values

In [ ]:
# X_test = test2.drop(columns=['category', 'preprocessed', 'reviews_content'])
# y_test = test2['category'].values
# s_test = pd.cut(test2['sentiment'], 3000, labels=False).values

In [ ]:
VALIDATION_SIZE = 0.2
SEED = 42

PERF_METRIC = "accuracy"
DISP_METRIC = "equalized_odds_diff"    # measures disparity, lower is better
FAIR_METRIC = "equalized_odds_ratio"   # similar but higher is better

N_JOBS = max(2, os.cpu_count() - 2)
np.random.seed(SEED)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_val, y_train, y_val, s_train, s_val = train_test_split(
#     X, Y, S,
#     test_size=VALIDATION_SIZE,
#     stratify=Y, random_state=SEED,
# )

In [ ]:
TRAIN_LENGTH = int(train_features.shape[0] * 0.8)
X_train = train_features.toarray()[:TRAIN_LENGTH, :]
X_val = train_features.toarray()[TRAIN_LENGTH:, :]
X_test = test2_features.toarray()

y_train = Y[:TRAIN_LENGTH]
y_val = Y[TRAIN_LENGTH:]
y_test = le.transform(test2['category'].values)

s_train = S[:TRAIN_LENGTH]
s_val = S[TRAIN_LENGTH:]
s_test = pd.cut(train['sentiment'], 3000, labels=False).values

### Find optimal parameter

In [ ]:
from hpt.tuner import ObjectiveFunction, OptunaTuner
import optuna.logging
optuna.logging.set_verbosity(optuna.logging.WARN)   # Turn off optuna log notes.

obj_func = ObjectiveFunction(
    X_train=X_train, y_train=y_train, s_train=s_train,
    X_val=X_val, y_val=y_val, s_val=s_val,
    X_test=X_test, y_test=y_test, s_test=s_test,
    hyperparameter_space=fairgbm_hyperparam_space_path,
    eval_metric=PERF_METRIC,
    other_eval_metric=FAIR_METRIC,
    threshold=0.50,
)

tuner = OptunaTuner(
    objective_function=obj_func,
    direction="maximize",
    seed=SEED,
)

### FairGBM Model

In [ ]:
from fairgbm import FairGBMClassifier

In [ ]:
core_lgbm_params = {
    "n_estimators": 200,
    "random_state": SEED,
    "n_jobs": -2,
}

fairgbm_clf = FairGBMClassifier(
    constraint_type="FNR",    # constraint on equal group-wise TPR (equal opportunity)
    multiplier_learning_rate=0.2,
    constraint_fnr_slack=0.05,
    **core_lgbm_params,
)

In [ ]:
fairgbm_clf.fit(train_features, Y, constraint_group=S)

FairGBMClassifier(constraint_fnr_slack=0.05, constraint_type='FNR',
                  multiplier_learning_rate=0.2, n_estimators=200, n_jobs=-2,
                  random_state=42)

In [ ]:
fairgbm_pred = le.inverse_transform(fairgbm_clf.predict(test2_features))

/usr/local/lib/python3.10/dist-packages/fairgbm/basic.py:741: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


In [ ]:
report = classification_report(test2['category'], fairgbm_pred, output_dict=True)

print('positive: ', report['positive'])
print('negative: ', report['negative'])

positive:  {'precision': 0.8928571428571429, 'recall': 0.10548523206751055, 'f1-score': 0.18867924528301885, 'support': 237}
negative:  {'precision': 0.5508474576271186, 'recall': 0.9885931558935361, 'f1-score': 0.7074829931972789, 'support': 263}


In [ ]:
fairgbm_final_pred = le.inverse_transform(fairgbm_clf.predict(test_features))

/usr/local/lib/python3.10/dist-packages/fairgbm/basic.py:741: UserWarning: Converting data to scipy sparse matrix.
  _log_warning('Converting data to scipy sparse matrix.')


# Voting

In [ ]:
def classify(pred):
  votes=[]
  confidences=[]
  for i in range(pred.shape[1]):
    curr = le.transform(pred[:, i])
    # pos = curr.sum()
    # neg = pred.shape[0] - pos
    choice_votes = mode(curr)
    confidence = choice_votes/len(curr)
    votes.append(choice_votes)
    confidences.append(confidence)

  return np.array(votes), np.array(confidences)

In [ ]:
final_pred, final_confidences = classify(all_pred)

In [ ]:
final_pred = le.inverse_transform(final_pred)

In [ ]:
output_files = pd.DataFrame({'Row': range(1, 501), 'Label': final_pred})
output_files

,Row,Label
0,1,positive
1,2,positive
2,3,negative
3,4,negative
4,5,positive
...,...,...
495,496,positive
496,497,positive
497,498,positive
498,499,positive


In [ ]:
output_files.to_csv('output_ensemble.csv', index=False)